In [35]:
#type:ignore
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import matplotlib as plt
from tensorflow.keras.applications import ResNet50, MobileNetV2  
from tensorflow.keras.models import Model
from tensorflow.keras import layers,optimizers
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
IMG_SIZE =224

In [36]:
df = pd.read_csv('labels.csv')

In [37]:
df.head()

,name,category,type,grade
0,1.jpg,0,4,3
1,2.jpg,0,4,3
2,3.jpg,0,2,2
3,4.jpg,0,2,1
4,5.jpg,0,1,3


In [38]:
image_paths = ['images/'+ name for name in df['name'].values]

In [39]:
y = df['type'].values

In [40]:
y_augmented = np.concatenate([y, y])

In [41]:
y_labels = to_categorical(y_augmented,num_classes=5)

In [42]:
y_labels

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [43]:
def load_image(image_path):
  img = cv2.imread(image_path)      # read image
  if img is None:
      print(f"Warning: Unable to load image at {image_path}")
      return None
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)       # convert to RGB
  img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))    # resize for the model
  img = img/255.0      # normalize pixel values to between 0 and 1

  return img

In [44]:
def rotate_image(image_path):
  img = cv2.imread(image_path)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)  # apply same preprocessing features as original image

  img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

  img = cv2.resize(img, (IMG_SIZE,IMG_SIZE)) 

  img = img/255.0

  return img

In [45]:
augmented_images = []

for path in image_paths:
    img = load_image(path)          # Original
    augmented_images.append(img)
    
    # Augment (rotate by 90 degrees)
    rotated_img = rotate_image(path)
    augmented_images.append(rotated_img)

# Convert to numpy array
X_augmented = np.array(augmented_images)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.3, random_state=42) # training and testing

In [47]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))  # loading model without top layers

In [48]:
base_model.trainable = False  # Freezing layers to exclude from training

In [49]:
#for layer in base_model.layers[-20:]:
    #layer.trainable = True

In [50]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

In [51]:
output_layer = Dense(5, activation='softmax', name='type')(x)

In [52]:
model = Model(inputs = base_model.input, outputs=output_layer)

In [53]:
model.compile(optimizer='adam', loss=
        'sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,587,205 (9.87 MB)

 Trainable params: 329,221 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [55]:
model.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_test, y_test), verbose=1)

Epoch 1/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 280ms/step - accuracy: 0.6626 - loss: 1.2859 - val_accuracy: 0.7734 - val_loss: 0.8154
Epoch 2/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 240ms/step - accuracy: 0.7530 - loss: 0.8771 - val_accuracy: 0.7734 - val_loss: 0.8446
Epoch 3/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 234ms/step - accuracy: 0.7694 - loss: 0.8035 - val_accuracy: 0.7734 - val_loss: 0.8332
Epoch 4/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 235ms/step - accuracy: 0.7695 - loss: 0.8085 - val_accuracy: 0.7734 - val_loss: 0.8497
Epoch 5/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 239ms/step - accuracy: 0.7773 - loss: 0.7345 - val_accuracy: 0.7734 - val_loss: 0.8410
Epoch 6/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 237ms/step - accuracy: 0.7851 - loss: 0.7722 - val_accuracy: 0.7734 - val_loss: 0.8380
Epoch 7/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 240ms/step - accuracy: 0.7630 - loss: 0.7475 - val_accuracy: 0.7734 - val_loss: 0.8563
Epoch 8/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 243ms/step - accuracy: 0.7920 - loss: 0.6348 - val_accuracy: 0

In [56]:
model.save('corneal_ulcer_model.h5')